# SFILES Parser Factory

Next we explore the potential of creating an SFILES parser factory. `sfilesParserFactory(types)` accepts a dictionary of process types and returns a parser function.

In [155]:
from pyparsing import Literal, Word, Group, Suppress
from pyparsing import Optional, OneOrMore, ZeroOrMore, oneOf, nestedExpr
from pyparsing import alphas, nums

types = {
    'i'    : {'name': 'input'},
    'o'    : {'name': 'output'},
    'f'    : {'name': 'flash'},
    'e'    : {'name': ''},
    'm'    : {'name': ''},
    'n'    : {'name': ''},
    'p'    : {'name': 'reactor product'},
    'cyc'  : {'name': 'solvent based azeotropic distillation'}, 
    'r'    : {'name': 'reactor'},
    'sw'   : {'name': 'pressure swing distillation'},
    'pms'  : {'name': 'polar molecule sieve based separation'},
    'ms'   : {'name': 'molecular sieve based separation'},
    'lmem' : {'name': 'liquid membrane based separation'},
    'gmem' : {'name': 'gas membrane based separation'},
    'crs'  : {'name': 'crystallization'},
    'ab'   : {'name': 'absorption'}
}

class ParseNode():
    
    def __init__(self, tokens):
        self.tokens = tokens
        self.assignFields()
        
    def __str__(self):
        return self.__class__.__name__ + ':' + str(self.__dict__)
    
    __repr__ = __str__  
    
    
class Component(ParseNode):
    
    def assignFields(self):
        self.name = self.tokens[0]
        
    def __str__(self):
        return self.__class__.__name__ + ':' + str(self.name)
    
    __repr__ = __str__
       
        
class Mixture(ParseNode):
    
    def assignFields(self):
        self.components = self.tokens[0]
        
    def __str__(self):
        return self.__class__.__name__ + ':' + str(self.components)
    
    __repr__ = __str__


class Stream(ParseNode):
    
    def assignFields(self):
        self.type = self.tokens[0][0]
        self.mixture = self.tokens[0][1]
        
    def __str__(self):
        return self.__class__.__name__ + ':' + str(self.type) + str(self.mixture)
    
    __repr__ = __str__


def sfilesParserFactory(types=types):
    LPAR  = Suppress("(")
    RPAR  = Suppress(")")
    LBRA  = Suppress("[")
    RBRA  = Suppress("]")
    SLASH = Suppress("/")
    GT = Literal(">")
    LT = Literal("<")

    # components
    component = Word(alphas.upper(), exact=1)
    component.setParseAction(Component)

    # mixtures
    mixture = Group(OneOrMore(component))
    mixture.setParseAction(Mixture)

    # first unit and stream in a process group
    type = Optional(oneOf(' '.join(types.keys())), default='dist')
    stream = Group(type + mixture)
    stream.setParseAction(Stream)

    # subsequent units and streams in a process group
    type_ = Optional(oneOf(' '.join(types.keys())), default='s')
    stream_ = Group(type_ + mixture)
    stream_.setParseAction(Stream)

    # process group
    processgroup = Group(LPAR + stream + ZeroOrMore(SLASH + stream_) + RPAR)

    # a process group sequence is comprised of connectors, process group, and recycles                                             
    connector = Optional(GT | LT, default=GT)
    recycle = Word(nums, exact=1)
    sequence = Group(processgroup + ZeroOrMore(connector + (processgroup | recycle )))

    # nested branches
    branchsequence = OneOrMore(connector + (processgroup | recycle ))
    branch = nestedExpr(opener=LBRA, closer=RBRA, content=branchsequence)

    # sfiles expression start with sequence
    sfiles = sequence + ZeroOrMore(branch | sequence)

    return sfiles.parseString

sfiles = sfilesParserFactory(types)

sfiles('(iA)(rAB/pABCD)<1<2[<(iB)](mABC/D)[<(oD)](A/BC)1(cycB/C)2(oC)').asList()

[[[Stream:iMixture:[Component:A]],
  ">",
  [Stream:rMixture:[Component:A, Component:B],
   Stream:pMixture:[Component:A, Component:B, Component:C, Component:D]],
  '<',
  '1',
  '<',
  '2'],
 ['<', [Stream:iMixture:[Component:B]]],
 [[Stream:mMixture:[Component:A, Component:B, Component:C],
   Stream:sMixture:[Component:D]]],
 ['<', [Stream:oMixture:[Component:D]]],
 [[Stream:distMixture:[Component:A],
   Stream:sMixture:[Component:B, Component:C]],
  ">",
  '1',
  ">",
  [Stream:cycMixture:[Component:B], Stream:sMixture:[Component:C]],
  ">",
  '2',
  ">",
  [Stream:oMixture:[Component:C]]]]